# Parameterizing ligand


In [1]:
from pyfoldx.handlers.fileHandler import FileHandler
from pyfoldx.structure import Structure
from pyfoldx.paramx import parameterize, JSonMolecule
import os

In [2]:
os.makedirs("paramx_output", exist_ok=True)

## **Step 1**: Extract ligand coordinates from the PDB

Load the structure and keep the lines of the ligand.

In [3]:
# Load your complex.pdb
structure = Structure("complex", "complex.pdb")

The *Structure* class has a *data* dictionary organized by chains, residues and atoms.

In [4]:
# Extract the ligand residue
lig_residue = None
for chain in structure.data:
    for res in structure.data[chain]:
        if structure.data[chain][res].code == "MFX":
            lig_residue = structure.data[chain][res]
            break
    if lig_residue:
        break

if not lig_residue:
    raise ValueError("❌ Ligand residue 'MFX' not found in complex.pdb")
lig_residue

Residue(MFX,1,L,True)

In [5]:
lig_lines = []
for chain in structure.data:
    for res in structure.data[chain]:
        if structure.data[chain][res].code == "MFX":
            lig_lines.extend(structure.data[chain][res].toPdb().split("\n"))

FileHandler.writeLines("paramx_output/ligand_full.pdb", lig_lines)

print(f"✅ Extracted {len(lig_lines)} lines for MFX.")

✅ Extracted 31 lines for MFX.


Isolate the MFX residue lines for parameterization

In [6]:
lig_lines

['HETATM 8024  N1  MFX L   1     215.805 228.704 201.575  1.00  0.00           N  ',
 'HETATM 8025  N2  MFX L   1     215.975 229.511 206.539  1.00  0.00           N  ',
 'HETATM 8026  N3  MFX L   1     215.408 229.263 210.212  1.00  0.00           N  ',
 'HETATM 8027  C1  MFX L   1     215.509 229.840 205.235  1.00  0.00           C  ',
 'HETATM 8028  C2  MFX L   1     215.497 229.470 202.768  1.00  0.00           C  ',
 'HETATM 8029  C3  MFX L   1     215.750 229.010 204.097  1.00  0.00           C  ',
 'HETATM 8030  C4  MFX L   1     214.962 230.792 202.612  1.00  0.00           C  ',
 'HETATM 8031  C5  MFX L   1     214.916 231.098 204.997  1.00  0.00           C  ',
 'HETATM 8032  C6  MFX L   1     215.566 229.520 200.400  1.00  0.00           C  ',
 'HETATM 8033  C7  MFX L   1     214.645 231.567 203.718  1.00  0.00           C  ',
 'HETATM 8034  C8  MFX L   1     215.131 230.689 200.070  1.00  0.00           C  ',
 'HETATM 8035  C9  MFX L   1     214.746 231.459 201.273  1.00  0

## **Step 2**: Mapping MFX atoms to FoldX parameters

Create atom mappings based on MFX's structure (modify according to your MFX's actual atoms):

### 🔬 PyFoldX ParamX Atom Typing Guide

This section defines the atom-type mapping used for PyFoldX ligand parameterization. You can either:

- Use this mapping table manually when creating your custom `paramx` JSON files, or  
- Use the provided `foldx_atomtyping.py` script, which applies these mappings automatically.

---

#### 🧾 Atom Type Mapping Table

| Generic Atom Type       | FoldX Atom Name | Residue |
|-------------------------|-----------------|---------|
| `O_hydroxyl`            | `OG`            | `SER`   |
| `O_ring`                | `O4*`           | `A`     |
| `O_minus`               | `OD1`           | `ASP`   |
| `O_carboxamide`         | `OD1`           | `ASN`   |
| `N_amino`               | `NZ`            | `LYS`   |
| `N_guanidino`           | `NH2`           | `ARG`   |
| `N_imidazol_plus`       | `ND1`           | `HIS`   |
| `N_imidazol_minus`      | `NE2`           | `HIS`   |
| `N_pyrazole`            | `N`             | `PRO`   |
| `N_amide`               | `ND2`           | `ASP`   |
| `C_ring_not_arom`       | `CG`            | `PRO`   |
| `C_ring_arom`           | `CZ`            | `PHE`   |
| `C_single_link`         | `CG2`           | `THR`   |
| `C_double_link`         | `CG`            | `ARG`   |
| `C_triple_link`         | `CG`            | `LEU`   |

---


In [7]:
atomMappingsDict = {
    "N1": "N_amide",
    "N2": "N_amide",
    "N3": "N_pyrazole",
    "C1": "C_ring_arom",
    "C2": "C_ring_arom",
    "C3": "C_ring_arom",
    "C4": "C_ring_arom",
    "C5": "C_ring_arom",
    "C6": "C_ring_arom",
    "C7": "C_ring_arom",
    "C8": "C_ring_arom",
    "C9": "C_ring_arom",
    "C10": "C_ring_not_arom",
    "C11": "C_ring_not_arom",
    "C12": "C_ring_not_arom",
    "C13": "C_ring_not_arom",
    "C14": "C_ring_not_arom",
    "C15": "C_ring_not_arom",
    "C16": "C_ring_not_arom",
    "C17": "C_single_link",
    "C18": "C_ring_not_arom",
    "C19": "C_ring_not_arom",
    "C20": "C_ring_not_arom",
    "C21": "C_double_link",
    "O1": "O_hydroxyl",
    "O2": "O_hydroxyl",
    "O3": "O_hydroxyl",
    "O4": "O_hydroxyl",
    "F1": "O_ring",
}

## **Step 3**: Create MFX parameter file

Generate the JSON parameter file for FoldX:

In [8]:
newMol = parameterize(lig_lines, atomMappingsDict)

Mappings loaded:
Atom N1 mapped to atom ('ND2', 'ASP')
Atom N2 mapped to atom ('ND2', 'ASP')
Atom N3 mapped to atom ('N', 'PRO')
Atom C1 mapped to atom ('CZ', 'PHE')
Atom C2 mapped to atom ('CZ', 'PHE')
Atom C3 mapped to atom ('CZ', 'PHE')
Atom C4 mapped to atom ('CZ', 'PHE')
Atom C5 mapped to atom ('CZ', 'PHE')
Atom C6 mapped to atom ('CZ', 'PHE')
Atom C7 mapped to atom ('CZ', 'PHE')
Atom C8 mapped to atom ('CZ', 'PHE')
Atom C9 mapped to atom ('CZ', 'PHE')
Atom C10 mapped to atom ('CG', 'PRO')
Atom C11 mapped to atom ('CG', 'PRO')
Atom C12 mapped to atom ('CG', 'PRO')
Atom C13 mapped to atom ('CG', 'PRO')
Atom C14 mapped to atom ('CG', 'PRO')
Atom C15 mapped to atom ('CG', 'PRO')
Atom C16 mapped to atom ('CG', 'PRO')
Atom C17 mapped to atom ('CG2', 'THR')
Atom C18 mapped to atom ('CG', 'PRO')
Atom C19 mapped to atom ('CG', 'PRO')
Atom C20 mapped to atom ('CG', 'PRO')
Atom C21 mapped to atom ('CG', 'ARG')
Atom O1 mapped to atom ('OG', 'SER')
Atom O2 mapped to atom ('OG', 'SER')
Atom O3

In [9]:
FileHandler.writeLine("molecules/MFX.json", newMol.toJson())

True